# Testing Data Set: Data Import & Cleaning

The testing data is kept in a separate .csv file from the training data, and thus was not included in initial cleaning procedures. In order to make accurate predictions on the testing data, I imported the data set here and performed the exact same cleaning operations as on the training data. Since the two data sets have identical columns (with the exception of the 'SalePrice' column, which is not included in the test set), I was able to simply execute the same exact code without having to debug. Upon completion of this operation, I exported it to a new .csv and will use it later on for modeling.

### Importing Packages and Data

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math as m
import datetime as dt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score

%matplotlib inline


df = pd.read_csv('./datasets/test.csv')
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


# Replacing Null Values

In [22]:
df.isnull().sum().sort_values(ascending=False)[:27]

Pool QC           875
Misc Feature      838
Alley             821
Fence             707
Fireplace Qu      422
Lot Frontage      160
Garage Cond        45
Garage Qual        45
Garage Yr Blt      45
Garage Finish      45
Garage Type        44
Bsmt Exposure      25
BsmtFin Type 1     25
Bsmt Qual          25
BsmtFin Type 2     25
Bsmt Cond          25
Mas Vnr Area        1
Mas Vnr Type        1
Electrical          1
Overall Cond        0
Exter Qual          0
Exter Cond          0
Foundation          0
Exterior 2nd        0
Exterior 1st        0
Roof Matl           0
Roof Style          0
dtype: int64

In [23]:
# Replacing nulls with 'NA' for all categorical data:

na_columns = ['Pool QC', 'Misc Feature', 'Alley', 'Fence', 'Fireplace Qu', 'Garage Finish', 'Garage Cond', 
              'Garage Qual', 'Garage Type', 'Bsmt Exposure', 'BsmtFin Type 2', 'BsmtFin Type 2', 'Bsmt Cond', 
              'Bsmt Qual', 'Mas Vnr Type'] 

[df[x].replace(np.nan, 'NA', inplace=True) for x in na_columns]

df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NA,NA,NA,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,NA,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,NA,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,NA,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NA,IR1,Lvl,...,0,185,0,NA,NA,NA,0,7,2009,WD


In [24]:
# Converting nulls to 0.0 for numeric data:

zero_columns = ['Lot Frontage', 'Garage Yr Blt', 'Mas Vnr Area', 'Bsmt Half Bath', 'Bsmt Full Bath', 
                'Garage Cars', 'Garage Area', 'Bsmt Unf SF', 'BsmtFin SF 2', 'Total Bsmt SF', 'BsmtFin SF 1']

[df[x].replace(np.nan, '0.0', inplace=True) for x in zero_columns]

df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NA,NA,NA,0,4,2006,WD
1,2718,905108090,90,RL,0.0,9662,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,NA,0,8,2006,WD
2,2414,528218130,60,RL,58,17104,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,NA,0,9,2006,New
3,1989,902207150,30,RM,60,8520,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,NA,0,7,2007,WD
4,625,535105100,20,RL,0.0,9500,Pave,NA,IR1,Lvl,...,0,185,0,NA,NA,NA,0,7,2009,WD


In [25]:
# Converting all "None" values to 'NA':

df.replace('None', 'NA', inplace=True)
df['Mas Vnr Type'].unique()

array(['NA', 'BrkFace', 'Stone', 'BrkCmn', 'CBlock'], dtype=object)

# Cleaning

In [26]:
# Lot frontage, Mas Vnr Area, Garage Cars, and Garage Area should be columns of numeric values. 
# I'll change these columns to floats:

floatify = ['Lot Frontage', 'Mas Vnr Area', 'Garage Area', 'Garage Cars']

for x in floatify:
    df[x] = df[x].astype('float64', copy=False)

In [27]:
# Consolidating bathrooms

baths = [x for x in df.columns if 'Bath' in x]

df['Total Baths'] = df[baths].sum(axis=1)
df.drop(['Full Bath', 'Half Bath', 'Bsmt Full Bath', 'Bsmt Half Bath'], axis=1, inplace=True)

In [28]:
# Creating 'Has Finished Garage' column for Finished or Rough Finished garages, and 'Has Detached Garage' column

searchfor = 'Fin', 'RFn'
df['Has Finished Garage'] = np.where(df['Garage Finish'].str.contains('|'.join(searchfor)), 1, 0)

df['Detached Garage'] = np.where(df['Garage Type'].str.contains('Detchd'), 1, 0)

df.drop(['Garage Finish', 'Garage Yr Blt', 'Garage Type',], axis=1, inplace=True)

In [29]:
# Consolidating basement columns into 'Has Finished Basement'

lookfor = 'GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ'

df['Has Finished Basement'] = np.where(df['BsmtFin Type 2'].str.contains('|'.join(lookfor)), 1, 0)
df['Has Finished Basement'] = np.where(df['BsmtFin Type 1'].str.contains('|'.join(lookfor)), 1, 0)


df.drop(['BsmtFin Type 1', 'BsmtFin Type 2', 'Bsmt Cond', 'Bsmt Exposure'], axis=1, inplace=True)

In [30]:
# Creating a new column of total square feet

sq_ft = [x for x in df.columns if 'SF' in x]

df['Total sq ft'] = df[sq_ft].sum(axis=1)

In [31]:
# Converting 'Year Remod/Add' to numerical 'Is Remodeled'

df['Is Remodeled'] = np.where(df['Year Remod/Add'] - df['Year Built'] > 0, 1, 0)
df.drop(['Year Remod/Add'], axis=1, inplace=True)

In [32]:
# Pool

df['Has Pool'] = np.where(df['Pool QC'].str.contains('NA'), 0, 1)
df.drop(['Pool QC'], axis=1, inplace=True)

In [33]:
# Fence

df['Has Fence'] = np.where(df['Fence'].str.contains('NA'), 0, 1)
df.drop(['Fence'], axis=1, inplace=True)

In [34]:
# Paved Drive

df['Has Paved Drive'] = np.where(df['Paved Drive'].str.contains('N'), 0, 1)
df.drop(['Paved Drive'], axis=1, inplace=True)

In [35]:
# Central Air

df['Has Central Air'] = np.where(df['Central Air'].str.contains('N'), 0, 1)
df.drop(['Central Air'], axis=1, inplace=True)

In [36]:
# Created dummy columns

df = pd.get_dummies(df, columns = ['Lot Config', 'MS Zoning', 'Misc Feature', 'Heating QC', 
                                   'House Style', 'Bldg Type'], drop_first=True)

In [37]:
# Quality rankings - transforming categorical vairables to ordinal ones

cols = ['Bsmt Qual', 'Fireplace Qu', 'Garage Qual', 'Garage Cond', 'Exter Qual', 
        'Exter Cond', 'Kitchen Qual', 'Exter Qual', 'Exter Cond']

rankings = {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}

for x in cols:
    df[x] = df[x].map(lambda x: rankings.get(x, x))

In [38]:
# Turning the 'Year Built' column into 'Age'

now = dt.datetime.now()
df['Current Year'] = now.year
df['Age'] = df['Current Year'] - df['Year Built']
df.drop(['Current Year', 'Year Built', 'Mo Sold', 'Yr Sold'], axis=1, inplace=True)

In [39]:
# Dropping the remaining object columns

df.drop(df.select_dtypes(include=['object']).columns, axis=1, inplace=True)

In [40]:
# Saving the cleaned data to a .csv file 

df.to_csv('./datasets/test_v2.csv', index=False)